<a href="https://colab.research.google.com/github/dquangucsd/Lab2_Starter/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -L -o ./classical-music-midi.zip\
  https://www.kaggle.com/api/v1/datasets/download/soumikrakshit/classical-music-midi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2388k  100 2388k    0     0  2582k      0 --:--:-- --:--:-- --:--:-- 2582k


In [2]:
!ls midis

ls: cannot access 'midis': No such file or directory


In [3]:
!mkdir midis

In [4]:
!ls midis

In [5]:
!mv classical-music-midi.zip midis/

In [6]:
!ls midis

classical-music-midi.zip


In [7]:
!unzip ./midis/classical-music-midi.zip

Archive:  ./midis/classical-music-midi.zip
  inflating: albeniz/alb_esp1.mid    
  inflating: albeniz/alb_esp2.mid    
  inflating: albeniz/alb_esp3.mid    
  inflating: albeniz/alb_esp4.mid    
  inflating: albeniz/alb_esp5.mid    
  inflating: albeniz/alb_esp6.mid    
  inflating: albeniz/alb_se1.mid     
  inflating: albeniz/alb_se2.mid     
  inflating: albeniz/alb_se3.mid     
  inflating: albeniz/alb_se4.mid     
  inflating: albeniz/alb_se5.mid     
  inflating: albeniz/alb_se6.mid     
  inflating: albeniz/alb_se7.mid     
  inflating: albeniz/alb_se8.mid     
  inflating: bach/bach_846.mid       
  inflating: bach/bach_847.mid       
  inflating: bach/bach_850.mid       
  inflating: balakir/islamei.mid     
  inflating: beeth/appass_1.mid      
  inflating: beeth/appass_2.mid      
  inflating: beeth/appass_3.mid      
  inflating: beeth/beethoven_hammerklavier_1.mid  
  inflating: beeth/beethoven_hammerklavier_2.mid  
  inflating: beeth/beethoven_hammerklavier_3.mid  
  infl

In [8]:
from glob import glob

In [9]:
midi_files = glob('chopin/*.mid')

In [10]:
len(midi_files)

48

In [11]:
!pip install tensorflow


In [12]:
!pip install music21

In [13]:
!pip install seaborn

In [14]:
!pip install pydot

In [15]:
#Importing Libraries
import tensorflow
import numpy as np
import pandas as pd
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax, Adam
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [16]:
!which lilypond

In [17]:
import os

In [18]:
from tqdm import tqdm

In [ ]:
#Loading the list of chopin's midi files as stream
filepath = "./chopin/"
#Getting midi files
all_midis= []
for i in tqdm(os.listdir(filepath)):
    if i.endswith(".mid"):
        tr = filepath+i
        try:
            midi = converter.parse(tr)
        except Exception as e:
            print(f"Error parsing {tr}: {e}")
            continue
        all_midis.append(midi)

 75%|███████▌  | 36/48 [02:00<00:41,  3.43s/it]

In [ ]:
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

In [ ]:
print("First fifty values in the Corpus:", Corpus[:50])


In [ ]:
# #First Lets write some functions that we need to look into the data
# def show(music):
#     display(Image(str(music.write("lily.png"))))

# def chords_n_notes(Snippet):
#     Melody = []
#     offset = 0 #Incremental
#     for i in Snippet:
#         #If it is chord
#         if ("." in i or i.isdigit()):
#             chord_notes = i.split(".") #Seperating the notes in chord
#             notes = []
#             for j in chord_notes:
#                 inst_note=int(j)
#                 note_snip = note.Note(inst_note)
#                 notes.append(note_snip)
#                 chord_snip = chord.Chord(notes)
#                 chord_snip.offset = offset
#                 Melody.append(chord_snip)
#         # pattern is a note
#         else:
#             note_snip = note.Note(i)
#             note_snip.offset = offset
#             Melody.append(note_snip)
#         # increase offset each iteration so that notes do not stack
#         offset += 1
#     Melody_midi = stream.Stream(Melody)
#     return Melody_midi

In [ ]:
# Melody_Snippet = chords_n_notes(Corpus[:100])
# show(Melody_Snippet)

In [ ]:
from music21 import *

# Printing the music sheet and playing MIDI
def show_and_play(music):
    # Show the music sheet as an image
    display(Image(str(music.write("lily.png"))))

    # Play the music using MIDI
    music.show('midi')  # This will play the music using the default MIDI player

def chords_n_notes(Snippet):
    Melody = []
    offset = 0
    for i in Snippet:
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".")  # Breaking down the notes that make up a chord
            notes = []
            for j in chord_notes:
                inst_note = int(j)
                note_snip = note.Note(inst_note)
                notes.append(note_snip)
            chord_snip = chord.Chord(notes)
            chord_snip.offset = offset
            Melody.append(chord_snip)
        else:
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        offset += 1

    Melody_midi = stream.Stream(Melody)
    return Melody_midi


In [ ]:
!apt update
!apt install -y lilypond

In [ ]:
from music21 import environment
us = environment.UserSettings()
print(us.getSettingsPath())
print(us['lilypondPath'])

In [ ]:
# from music21 import environment
# us = environment.UserSettings()
# # Now set the path (replace with your actual path)
# us['lilypondPath'] = '/usr/bin/lilypond' # Or your specific path


In [ ]:

# Example: Create a Melody from a snippet of the Corpus
Melody_Snippet = chords_n_notes(Corpus[:100])

# Show the music sheet and play the music
show_and_play(Melody_Snippet)

In [ ]:
count_num = Counter(Corpus)
print("\033[96m The total count of unique notes in the Corpus:\033[0m", len(count_num))

In [ ]:
Notes = list(count_num.keys())
Recurrence = list(count_num.values())
# Plotting the distribution of Notes with enhanced aesthetics
plt.figure(figsize=(18, 6))
bins = np.arange(0, max(Recurrence) + 50, 50)
plt.hist(Recurrence, bins=bins, color="#86A9C1", edgecolor="#5D6F82", alpha=0.7)
plt.axvline(x=100, color="#DBACC1", linestyle='--', linewidth=2, label='Threshold: 100')
plt.title("Frequency Distribution of Notes in the Corpus", fontsize=16, weight='bold')
plt.xlabel("Number of Occurrences per Note", fontsize=12)
plt.ylabel("Number of Unique Notes", fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100:
        m =  key
        rare_note.append(m)

print("\033[96m The total count of notes that appear fewer than 100 times:\033[0m", len(rare_note))

for element in Corpus:
    if element in rare_note:
        Corpus.remove(element)

print("\033[96m The length of the Corpus after removing the rare notes:\033[0m", len(Corpus))

In [ ]:
sym = sorted(list(set(Corpus)))
corp = len(Corpus) #length of corpus
symb = len(sym) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(sym))
reverse_mapping = dict((i, c) for i, c in enumerate(sym))

print("\033[96mTotal number of characters:\033[0m", corp)
print("\033[96mTotal number of unique characters:\033[0m", sym)


In [ ]:
length = 40
features = []
targets = []
for i in range(0, corp - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
data_ = len(targets)
print("\033[96mTotal count of sequences within the corpus:\033[0m", data_)

In [ ]:
# Normalize
X = (np.reshape(features, (data_, length, 1))) / float(len(sym))
y = tensorflow.keras.utils.to_categorical(targets)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model
model = Sequential()

# Bidirectional LSTM Layer 1
model.add(Bidirectional(LSTM(512, return_sequences=True, recurrent_dropout=0.2),
                        input_shape=(X.shape[1], X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# LSTM Layer 2
model.add(LSTM(256, return_sequences=True, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# LSTM Layer 3
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fully Connected Dense Layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(y.shape[1], activation='softmax'))

# Compile model
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.summary()


In [ ]:
plot_model(model, to_file='model_structure.png', show_shapes=True, show_layer_names=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size=256, epochs=50)


In [ ]:
history_df = pd.DataFrame(history.history)
sns.set_theme(style="whitegrid")
fig = plt.figure(figsize=(15, 8),)
fig.suptitle("Training Loss Over Epochs", fontsize=16, fontweight='bold')
ax = sns.lineplot(
    data=history_df["loss"],
    linewidth=2.5,
    color="#5A4E7C",
    marker='o',
    markersize=6
)
ax.set_xlabel("Epochs", fontsize=14, fontweight='semibold', color="#444444")
ax.set_ylabel("Training Loss", fontsize=14, fontweight='semibold', color="#444444")
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
from IPython.display import Image, display

def chords_n_notes(Snippet):
    Melody = []
    offset = 0
    for i in Snippet:
        if "." in i or i.isdigit():
            chord_notes = i.split(".")
            notes = []
            for j in chord_notes:
                inst_note = int(j)
                note_snip = note.Note(inst_note)
                notes.append(note_snip)
            chord_snip = chord.Chord(notes)
            chord_snip.offset = offset
            Melody.append(chord_snip)
        else:
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        offset += 1
    return stream.Stream(Melody)

def show_and_play(music):
    display(Image(str(music.write("lily.png"))))
    music.show('midi')

def Generator(count):
    seed_idx = np.random.randint(0, len(X_test) - 1)
    test = X_test[seed_idx]
    generated_indices = []

    for _ in range(count):
        test_input = test.reshape(1, length, 1)
        prediction = model.predict(test_input, verbose=0)[0]

        # Sampling with temperature
        prediction = np.log(prediction + 1e-8) / 1.0
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)

        # Sampling instead of argmax to create more variation
        index = np.random.choice(len(prediction), p=prediction)

        generated_indices.append(index)

        # Normalize new input and shift window
        next_note = index / float(len(sym))
        test = np.append(test, [[next_note]], axis=0)
        test = test[1:]

    # Map back to note names
    generated_notes = [reverse_mapping[i] for i in generated_indices]

    # Convert notes to MIDI stream
    melody = chords_n_notes(generated_notes)
    show_and_play(melody)

    return generated_notes, melody
music_notes, melody_stream = Generator(100)

In [ ]:
for i in range(5):
    print(f"\033[94m🎼 Melody Sample {i+1}\033[0m")
    notes, melody = Generator(100)
    show_and_play(melody)